#### Implement the baseline algos here

In [1]:
import sys; sys.path.insert(0, "../"); from utils import *
import pickle

%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from pde_diff import *
import robust_PDE_FIND
from robust_PDE_FIND import TrainSTRidge, compute_err, print_pde, RobustPCA, Robust_LRSTR
import scipy.io as sio

Running Python 3.9.8
You can use npar for np.array
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['diff', 'save']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
# Loading the KS sols
# data = sio.loadmat("../R_DLGA_datasets/KS.mat")
data = sio.loadmat("../PDE_FIND_experimental_datasets/kuramoto_sivishinky.mat")

t = data['tt'][0,:]
x = data['x'][:,0]

idx_x = np.arange(len(x))
# idx_x = idx_x[idx_x%1==0]
idx_t = np.arange(len(t))
# idx_t = idx_t[t<=7.6]

x = x[idx_x]
t = t[idx_t]

time_dims = t.shape[0]
spatial_dims = x.shape[0]

total_points = time_dims*spatial_dims
print("Total data points:", total_points)

X, T = np.meshgrid(x, t)
Exact = data['uu']
u = Exact[idx_x][:, idx_t]

# DATA_PATH = "../deephpms_data/KS_new3.pkl"
# data = pickle_load(DATA_PATH)
# t = data['t']
# x = data['x']
# X, T = np.meshgrid(x, t)
# Exact = data['u']
# u = Exact

# from rkstiff import grids
# from rkstiff import if34

# # Computing the KS sol
# # uniform grid spacing, real-valued u -> construct_x_kx_rfft
# N = 256
# # a, b = 0, 16*np.pi
# M = 2
# a, b = -M*np.pi, M*np.pi
# # a, b = -M, M
# x, kx = grids.construct_x_kx_rfft(N, a, b)

# L = kx**2*(1-kx**2)
# def NL(uFFT):
#     u = np.fft.irfft(uFFT)
#     ux = np.fft.irfft(1j*kx*uFFT)
#     return -np.fft.rfft(u*ux)

# u0 = -np.sin(np.pi*x/10)
# # u0 = np.cos(x/16)*(1.+np.sin(x/16))
# u0FFT = np.fft.rfft(u0)
# solver = if34.IF34(linop=L,NLfunc=NL)
# ufFFT = solver.evolve(u0FFT, t0=0, tf=10) # store every 20th step in solver.u and solver.t

# U = []
# for uFFT in solver.u:
#     U.append(np.fft.irfft(uFFT))
# U = np.array(U)
# t = np.array(solver.t)

# X, T = np.meshgrid(x, t)
# Exact = U.T
# u = Exact

# spatial_dims = Exact.shape[0]
# time_dims = Exact.shape[1]

Total data points: 257024


In [3]:
# fig1 = figure()
# ax = fig1.gca(projection='3d')
# surf = ax.plot_surface(X, T, u.T, rstride=1, cstride=1, cmap=cm.coolwarm,
#     linewidth=0, antialiased=False)
# title('Kuramoto Sivashinsky Equation', fontsize = 20)
# xlabel('x', fontsize = 16)
# ylabel('t', fontsize = 16)

#### STR

In [4]:
dt = t[1]-t[0]
dx = x[2]-x[1]
Ut, R, rhs_des = build_linear_system(u, dt, dx, D=5, P=5, time_diff = 'FD', space_diff = 'FD')

In [5]:
idx = np.random.choice(R.shape[0], R.shape[0], replace=False)

In [6]:
w = TrainSTRidge(R, Ut, 10**-5, 5)
# TrainSTRidge(X_input[:, :], y_input, 1e-6, 10, normalize=1)
# w = TrainSTRidge(R[idx, :],Ut[idx, :],10**-6,50)
print("PDE derived using STRidge")
print_pde(w, rhs_des)

/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_small_KS2/robust_PDE_FIND.py:845: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_small_KS2/robust_PDE_FIND.py:890: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_small_KS2/robu

PDE derived using STRidge
u_t = (-0.984425 +0.000000i)uu_{x}
    + (-0.994175 +0.000000i)u_{xx}
    + (-0.998730 +0.000000i)u_{xxxx}
   


In [7]:
print(u.shape)
total_points = u.shape[0]*u.shape[1]
print(total_points)

(1024, 251)
257024


In [8]:
# 1024*89 -> sample 256*89 to train

In [9]:
# with open("../deephpms_data/KS_new3.pkl", "wb") as f:
#     pickle.dump(dict({'x':x, 't':t, 'u':u}), f)

In [8]:
# PDE derived using STRidge
# u_t = (-0.984425 +0.000000i)uu_{x}
#     + (-0.994175 +0.000000i)u_{xx}
#     + (-0.998730 +0.000000i)u_{xxxx}

errs = 100*(np.abs(npar([-0.984425, -0.994175, -0.998730])+1))
errs.mean(), errs.std()

(0.7556666666666656, 0.5966979042102376)

#### STR + noise

In [10]:
noise_intensity = 0.01
numpy.random.seed(0)
un = u + noise_intensity*std(u)*np.random.randn(u.shape[0],u.shape[1])

In [11]:
# Utn, Rn, rhs_des = build_linear_system(un, dt, dx, D=5, P=5, time_diff = 'FD', space_diff = 'FD')

Utn, Rn, rhs_des = build_linear_system(un, dt, dx, D=5, P=5, 
                                 time_diff = 'poly', space_diff = 'poly', 
                                 width_x = 20, width_t = 20, deg_x = 6)
# Solve with STRidge
w = TrainSTRidge(Rn,Utn,10**-5,5)
print("PDE derived using STRidge")
print_pde(w, rhs_des)

PDE derived using STRidge
u_t = (-0.458521 +0.000000i)uu_{x}
    + (-0.480667 +0.000000i)u_{xx}
    + (-0.492282 +0.000000i)u_{xxxx}
   


In [12]:
# PDE derived using STRidge
# u_t = (-0.458521 +0.000000i)uu_{x}
#     + (-0.480667 +0.000000i)u_{xx}
#     + (-0.492282 +0.000000i)u_{xxxx}

In [13]:
errs = 100*(np.abs(npar([-0.458521, -0.480667, -0.492282])+1))
errs.mean(), errs.std()

(52.284333333333336, 1.4004597110314283)

#### DLrSR

In [40]:
u_in = u

Z, E1 = RobustPCA(u_in, lam_2 = 0.3)

diff_type = "fd"
if diff_type == "poly":
    ZUtn, ZRn, rhs_des = build_linear_system(Z, dt, dx, D=4, P=4, 
                                     time_diff = 'poly', space_diff = 'poly', 
                                     width_x = 20, width_t = 20, deg_x = 6)
elif diff_type == "fd":
    ZUtn, ZRn, rhs_des = build_linear_system(Z, dt, dx, D=4, P=4, time_diff = 'FD', space_diff = 'FD')

iteration:1, err:6782.698553354604, nc_norm:3140.707575226815 eta1:0.002541325899241079
iteration:50, err:0.08699106913499305, nc_norm:2762.854145694224 eta1:0.27120764977924544
iteration:100, err:0.0009321212448607521, nc_norm:1753.509475829864 eta1:31.837297315083465
iteration:149, err:9.457973655488348e-06, nc_norm:1744.918801098443 eta1:3397.643168365541


In [41]:
ZUtn.shape

(257024, 1)

In [42]:
if diff_type == "poly":
    double_width = 0
    for n in range(0, 101):
        if ZUtn.shape[0] == (spatial_dims-n)*(time_dims-n):
            double_width = n
            nx = spatial_dims-n
            nt = time_dims-n
            print(double_width); break
elif diff_type == "fd":
    (nx,nt) = spatial_dims, time_dims

In [43]:
# Solve with DLrSR
Utn1 = np.reshape(ZUtn, (nx,nt))
w, X, E2 = Robust_LRSTR(ZRn, Utn1, rhs_des, lam_1 = 1e-5, lam_3 = 0.3, lam_4 = 1e-4, d_tol = 5)
print_pde(w, rhs_des)

iteration:1, err:1499.9450715276414, nc_norm:807.220406170525 eta2:0.012510157726562877
u_t = 
u_t = (-0.984022 +0.000000i)uu_{x}
    + (-0.993733 +0.000000i)u_{xx}
    + (-0.998291 +0.000000i)u_{xxxx}
   
u_t = (-0.984573 +0.000000i)uu_{x}
    + (-0.994294 +0.000000i)u_{xx}
    + (-0.998854 +0.000000i)u_{xxxx}
   
u_t = (-0.984337 +0.000000i)uu_{x}
    + (-0.994060 +0.000000i)u_{xx}
    + (-0.998618 +0.000000i)u_{xxxx}
   
u_t = (-0.984345 +0.000000i)uu_{x}
    + (-0.994072 +0.000000i)u_{xx}
    + (-0.998630 +0.000000i)u_{xxxx}
   
u_t = (-0.984352 +0.000000i)uu_{x}
    + (-0.994083 +0.000000i)u_{xx}
    + (-0.998640 +0.000000i)u_{xxxx}
   
u_t = (-0.984358 +0.000000i)uu_{x}
    + (-0.994093 +0.000000i)u_{xx}
    + (-0.998650 +0.000000i)u_{xxxx}
   
u_t = (-0.984365 +0.000000i)uu_{x}
    + (-0.994102 +0.000000i)u_{xx}
    + (-0.998659 +0.000000i)u_{xxxx}
   
u_t = (-0.984371 +0.000000i)uu_{x}
    + (-0.994112 +0.000000i)u_{xx}
    + (-0.998668 +0.000000i)u_{xxxx}
   
u_t = (-0.984378 

In [9]:
errs = np.abs(100*(npar([-0.984412, -0.994159, -0.998715])+1))
errs.mean(), errs.std()

(0.7571333333333338, 0.5965986497544998)

#### DLrSR + noise (Normal distributed noise)

In [67]:
u_in = un
Z, E1 = RobustPCA(u_in, lam_2 = 0.1)
diff_type = "poly"
if diff_type == "poly":
    ZUtn, ZRn, rhs_des = build_linear_system(Z, dt, dx, D=4, P=4, 
                                     time_diff = 'poly', space_diff = 'poly', 
                                     width_x = 20, width_t = 20, deg_x = 6)
elif diff_type == "fd":
    ZUtn, ZRn, rhs_des = build_linear_system(Z, dt, dx, D=5, P=5, time_diff = 'FD', space_diff = 'FD')

iteration:1, err:6779.3651218702325, nc_norm:3267.101964367483 eta1:0.0025412325604941247
iteration:50, err:0.830026185886509, nc_norm:2831.9579896728214 eta1:0.2711976887655077
iteration:84, err:7.473817135667703e-06, nc_norm:1854.7548052580946 eta1:6.928469019885169


In [68]:
if diff_type == "poly":
    double_width = 0
    for n in range(0, 101):
        if ZUtn.shape[0] == (spatial_dims-n)*(time_dims-n):
            double_width = n
            nx = spatial_dims-n
            nt = time_dims-n
            print(double_width); break
elif diff_type == "fd":
    (nx,nt) = spatial_dims, time_dims

40


In [69]:
# Solve with DLrSR
Utn1 = np.reshape(ZUtn, (nx,nt))
w, X, E2 = Robust_LRSTR(ZRn, Utn1, rhs_des, lam_1 = 1e-5, lam_3 = 0.15, lam_4 = 1, d_tol = 5)
print_pde(w, rhs_des)

iteration:1, err:1083.0751108362106, nc_norm:686.6157329685841 eta2:0.015822935836742918
u_t = 
u_t = (-0.177664 +0.000000i)uu_{x}
    + (-0.114428 +0.000000i)u_{xx}
    + (-0.030660 +0.000000i)uu_{xxx}
    + (-0.145993 +0.000000i)u_{xxxx}
   
u_t = (-0.421044 +0.000000i)uu_{x}
    + (-0.263850 +0.000000i)u_{xx}
    + (-0.074821 +0.000000i)uu_{xxx}
    + (-0.341089 +0.000000i)u_{xxxx}
   
u_t = (-0.633930 +0.000000i)uu_{x}
    + (-0.396793 +0.000000i)u_{xx}
    + (-0.112763 +0.000000i)uu_{xxx}
    + (-0.513252 +0.000000i)u_{xxxx}
   
u_t = (-0.740830 +0.000000i)uu_{x}
    + (-0.467121 +0.000000i)u_{xx}
    + (-0.130753 +0.000000i)uu_{xxx}
    + (-0.602070 +0.000000i)u_{xxxx}
   
u_t = (-0.731976 +0.000000i)uu_{x}
    + (-0.466407 +0.000000i)u_{xx}
    + (-0.127736 +0.000000i)uu_{xxx}
    + (-0.598077 +0.000000i)u_{xxxx}
   
u_t = (-0.661025 +0.000000i)uu_{x}
    + (-0.425745 +0.000000i)u_{xx}
    + (-0.113985 +0.000000i)uu_{xxx}
    + (-0.543081 +0.000000i)u_{xxxx}
   
u_t = (-0.598243

In [70]:
# u_t = (-0.599524 +0.000000i)uu_{x}
#     + (-0.386087 +0.000000i)u_{xx}
#     + (-0.103480 +0.000000i)uu_{xxx}
#     + (-0.492624 +0.000000i)u_{xxxx}

In [71]:
# errs = np.abs(100*(npar([-0.784982, -0.813629, -0.808917])+1))
# errs.mean(), errs.std()

#### Notes on the AAAI'20 paper
    - The algo needs grid. Not possible to run the algo on freely distributed data points.
    - Assume strictly sparse-distributed noise.
    - It is not clear how to tune the lagrange multiplier hyper parameters so that the optimal true PDE is obtained.
    - The PDE solution and the pde diff features are not learned or updated joinly, hence, the sub-optimal equation is discovered, especially in the noisy case.